In [3]:
import depthai as dai
import cv2 as cv
import os
from datetime import datetime, timezone
from collections import deque
import math
import time

In [5]:


# ---- Config ----
FPS = 30
SYNC_TOLERANCE_MS = 8.0      # max timestamp difference allowed between streams
WAIT_SYNC_TIMEOUT_S = 0.5    # how long to wait to find a synced triplet after pressing space
BUFFER_LEN = 30              # ring buffer length per stream

# Create folders
base_dir = "images"
for sub in ["rgb", "left", "right"]:
    os.makedirs(os.path.join(base_dir, sub), exist_ok=True)

# ---- Build pipeline ----
pipeline = dai.Pipeline()

# RGB
cam_rgb  = pipeline.create(dai.node.ColorCamera)
xout_rgb = pipeline.create(dai.node.XLinkOut)
xout_rgb.setStreamName("rgb")

cam_rgb.setBoardSocket(dai.CameraBoardSocket.RGB)
cam_rgb.setResolution(dai.ColorCameraProperties.SensorResolution.THE_1080_P)
cam_rgb.setPreviewSize(416, 416)
cam_rgb.setInterleaved(False)
cam_rgb.setColorOrder(dai.ColorCameraProperties.ColorOrder.BGR)
cam_rgb.setFps(FPS)
#cam_rgb.setPreviewKeepAspectRatio(False)
cam_rgb.preview.link(xout_rgb.input)

# Mono Left
mono_left  = pipeline.create(dai.node.MonoCamera)
xout_left  = pipeline.create(dai.node.XLinkOut)
xout_left.setStreamName("left")

mono_left.setBoardSocket(dai.CameraBoardSocket.LEFT)
mono_left.setResolution(dai.MonoCameraProperties.SensorResolution.THE_400_P)
mono_left.setFps(FPS)
mono_left.out.link(xout_left.input)

# Mono Right
mono_right = pipeline.create(dai.node.MonoCamera)
xout_right = pipeline.create(dai.node.XLinkOut)
xout_right.setStreamName("right")

mono_right.setBoardSocket(dai.CameraBoardSocket.RIGHT)
mono_right.setResolution(dai.MonoCameraProperties.SensorResolution.THE_400_P)
mono_right.setFps(FPS)
mono_right.out.link(xout_right.input)

# ---- Helpers ----
def ts_ms(img_frame: dai.ImgFrame) -> float:
    # Use device-synchronized timestamp
    # getTimestampDevice() is most consistent across nodes
    t = img_frame.getTimestampDevice()
    return t.total_seconds() * 1000.0

def best_match(anchor_ts_ms: float, candidates: deque) -> tuple:
    """
    Find frame in candidates with timestamp closest to anchor_ts_ms.
    Returns (frame, dt_ms) or (None, inf) if empty.
    """
    best = (None, math.inf)
    for (t, f) in candidates:
        d = abs(t - anchor_ts_ms)
        if d < best[1]:
            best = (f, d)
    return best

def try_get_synced_triplet(q_rgb, q_left, q_right, buf_rgb, buf_left, buf_right) -> tuple:
    """
    Attempt to find a triplet such that timestamps are within SYNC_TOLERANCE_MS.
    Uses latest RGB as anchor; finds nearest Left/Right.
    Returns (rgb_frame, left_frame, right_frame) or (None,None,None)
    """
    if not buf_rgb:
        return (None, None, None)

    # Use the most recent RGB timestamp as anchor
    anchor_ts, rgb_frame = buf_rgb[-1]

    left_frame, dL = best_match(anchor_ts, buf_left)
    right_frame, dR = best_match(anchor_ts, buf_right)

    if left_frame is not None and right_frame is not None and dL <= SYNC_TOLERANCE_MS and dR <= SYNC_TOLERANCE_MS:
        return (rgb_frame, left_frame, right_frame)
    return (None, None, None)

# ---- Run ----
with dai.Device(pipeline) as device:
    q_rgb   = device.getOutputQueue("rgb",   maxSize=8, blocking=False)
    q_left  = device.getOutputQueue("left",  maxSize=8, blocking=False)
    q_right = device.getOutputQueue("right", maxSize=8, blocking=False)

    # Ring buffers of (timestamp_ms, frame)
    buf_rgb   = deque(maxlen=BUFFER_LEN)
    buf_left  = deque(maxlen=BUFFER_LEN)
    buf_right = deque(maxlen=BUFFER_LEN)

    # Latest frames for display
    disp_rgb = disp_left = disp_right = None

    while True:
        # Ingest new frames
        in_rgb = q_rgb.tryGet()
        if in_rgb:
            disp_rgb = in_rgb.getCvFrame()
            buf_rgb.append((ts_ms(in_rgb), disp_rgb))
            cv.imshow("RGB", disp_rgb)

        in_left = q_left.tryGet()
        if in_left:
            disp_left = in_left.getCvFrame()
            buf_left.append((ts_ms(in_left), disp_left))
            cv.imshow("Left", disp_left)

        in_right = q_right.tryGet()
        if in_right:
            disp_right = in_right.getCvFrame()
            buf_right.append((ts_ms(in_right), disp_right))
            cv.imshow("Right", disp_right)

        key = cv.waitKey(1) & 0xFF
        if key == ord('q'):
            break
        elif key == 32:  # Space pressed -> capture synchronized triplet
            start_wait = time.time()
            saved = False

            while time.time() - start_wait < WAIT_SYNC_TIMEOUT_S:
                # Keep draining queues so buffers stay fresh while we wait
                got_any = False
                pkt = q_rgb.tryGet()
                if pkt:
                    disp_rgb = pkt.getCvFrame()
                    buf_rgb.append((ts_ms(pkt), disp_rgb))
                    got_any = True
                pkt = q_left.tryGet()
                if pkt:
                    disp_left = pkt.getCvFrame()
                    buf_left.append((ts_ms(pkt), disp_left))
                    got_any = True
                pkt = q_right.tryGet()
                if pkt:
                    disp_right = pkt.getCvFrame()
                    buf_right.append((ts_ms(pkt), disp_right))
                    got_any = True

                rgb_f, left_f, right_f = try_get_synced_triplet(q_rgb, q_left, q_right, buf_rgb, buf_left, buf_right)
                if rgb_f is not None:
                    timestamp = datetime.now(timezone.utc).strftime("%Y%m%d_%H%M%S_%fZ")
                    rgb_path   = f"{base_dir}/rgb/rgb_{timestamp}.png"
                    left_path  = f"{base_dir}/left/left_{timestamp}.png"
                    right_path = f"{base_dir}/right/right_{timestamp}.png"
                    cv.imwrite(rgb_path, rgb_f)
                    cv.imwrite(left_path, left_f)
                    cv.imwrite(right_path, right_f)
                    print(f"Saved:\n  {rgb_path}\n  {left_path}\n  {right_path}")
                    saved = True
                    break

                # If nothing new arrived, tiny sleep to avoid busy-spin
                if not got_any:
                    time.sleep(0.002)

            if not saved:
                print("Warning: could not find tightly synchronized frames; try again or increase SYNC_TOLERANCE_MS / WAIT_SYNC_TIMEOUT_S.")

cv.destroyAllWindows()


C:\Users\david\AppData\Local\Temp\ipykernel_1844\2055005378.py:20: DeprecationWarning: RGB is deprecated, use CAM_A or address camera by name instead.
  cam_rgb.setBoardSocket(dai.CameraBoardSocket.RGB)
C:\Users\david\AppData\Local\Temp\ipykernel_1844\2055005378.py:34: DeprecationWarning: LEFT is deprecated, use CAM_B or address camera by name  instead.
  mono_left.setBoardSocket(dai.CameraBoardSocket.LEFT)
C:\Users\david\AppData\Local\Temp\ipykernel_1844\2055005378.py:44: DeprecationWarning: RIGHT is deprecated, use CAM_C or address camera by name  instead.
  mono_right.setBoardSocket(dai.CameraBoardSocket.RIGHT)


Saved:
  images/rgb/rgb_20251003_083454_559044Z.png
  images/left/left_20251003_083454_559044Z.png
  images/right/right_20251003_083454_559044Z.png
Saved:
  images/rgb/rgb_20251003_083456_763179Z.png
  images/left/left_20251003_083456_763179Z.png
  images/right/right_20251003_083456_763179Z.png
Saved:
  images/rgb/rgb_20251003_083458_558912Z.png
  images/left/left_20251003_083458_558912Z.png
  images/right/right_20251003_083458_558912Z.png
